In [13]:
from label_legends.preprocess import load_data, transform, majority, holdout_majority, load_test
import polars as pl
#deberta
import numpy as np
from polars import DataFrame, col
from transformers import AutoModelForSequenceClassification
from label_legends.deberta_majority import load_dataset, load_deberta, load_dataset
from label_legends.deberta import load_dataset, load_deberta, load_dataset
from label_legends.preprocess import holdout, load_data, load_test, transform
from label_legends.female import predict_female
from label_legends.result import calculate_scores

In [3]:
val, tra = holdout_majority()
val

/Users/retipeter/Documents/TU Wien/NLP/nlp-ie-label-legends/.venv/lib/python3.12/site-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


shape: (5, 6)
┌───────┬────────────────────┬────────────────────┬────────────────────┬───────┬───────────────────┐
│ id    ┆ text               ┆ tokens             ┆ token_ids          ┆ label ┆ rewire_id         │
│ ---   ┆ ---                ┆ ---                ┆ ---                ┆ ---   ┆ ---               │
│ i64   ┆ str                ┆ list[str]          ┆ list[i64]          ┆ i64   ┆ str               │
╞═══════╪════════════════════╪════════════════════╪════════════════════╪═══════╪═══════════════════╡
│ 0     ┆ I wonder what      ┆ ["wonder",         ┆ [2942, 2935, …     ┆ 1     ┆ sexism2022_englis │
│       ┆ keeps that witc…   ┆ "witch", … "😄"]   ┆ 3000]              ┆       ┆ h-0               │
│ 1     ┆ I wonder what      ┆ ["wonder",         ┆ [2942, 2935, …     ┆ 1     ┆ sexism2022_englis │
│       ┆ keeps that witc…   ┆ "witch", … "😄"]   ┆ 3000]              ┆       ┆ h-0               │
│ 10    ┆ Good for her! My   ┆ ["good", "!", …    ┆ [1181, 0, … 0]     ┆ 0     

rewire_id_number,label_sum,text,tokens,token_ids,label
i64,i64,str,list[str],list[i64],i32
17886,0,"""How do you make a Quija Board …","[""make"", ""quija"", … "".""]","[1648, 3000, … 26]",0
7045,0,"""watch what women do not what t…","[""watch"", ""woman"", ""say""]","[2889, 2939, 2331]",0
18448,1,"""A certain Muppet female pig ha…","[""certain"", ""muppet"", … ""#metoo""]","[508, 3000, … 3000]",0
18749,0,"""Katy Perry and her connection …","[""katy"", ""perry"", … "".""]","[3000, 3000, … 26]",0
15807,0,""" You got me! I'm really Soros'…","[""!"", ""really"", … ""blmforcash""]","[0, 2194, … 3000]",0
…,…,…,…,…,…
10674,0,"""Block/delete her from your pho…","[""block"", ""/"", … "".""]","[376, 35, … 26]",0
9724,3,"""These are accomplished women o…","[""accomplish"", ""woman"", … "".""]","[124, 2939, … 26]",1
8234,0,"""Word of The Hour catastrophe: …","[""word"", ""hour"", … ""catástrofe""]","[2945, 1318, … 3000]",0


In [4]:

tra

rewire_id_number,label_sum,text,tokens,token_ids,label
i64,i64,str,list[str],list[i64],i32
8527,0,"""This is just perfect! I love t…","[""just"", ""perfect"", … ""!""]","[1483, 1967, … 0]",0
11951,1,"""She is good, at least she is n…","[""good"", "","", … "".""]","[1181, 22, … 26]",0
14806,1,"""But of course, somehow the bla…","[""course"", "","", … "".""]","[682, 22, … 26]",0
8817,0,"""Idk man, maybe you leave her s…","[""idk"", ""man"", … ""reply""]","[1345, 1652, … 2238]",0
17475,3,"""back again hi I am sammay I li…","[""hi"", ""sammay"", … ""pest""]","[1272, 3000, … 3000]",1
…,…,…,…,…,…
18316,2,"""Hard of hearing chicks are nic…","[""hard"", ""hearing"", … "".""]","[1243, 3000, … 26]",1
6983,3,"""This actually makes me angry.I…","[""actually"", ""make"", … ""case""]","[143, 1648, … 493]",1
17348,3,"""the problem with this is that …","[""problem"", ""cuck"", … ""?""]","[2091, 712, … 98]",1


# Deberta running on majority here

In [5]:
deberta = load_deberta()

Some weights of DebertaV2ForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-v3-base and are newly initialized: ['classifier.bias', 'classifier.weight', 'pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
train_out = deberta.train()
train_out.metrics

  0%|          | 0/3065 [00:00<?, ?it/s]/Users/retipeter/Documents/TU Wien/NLP/nlp-ie-label-legends/src/label_legends/deberta_majority.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
  3%|▎         | 100/3065 [00:58<30:21,  1.63it/s]

{'loss': 0.5901, 'grad_norm': 0.8738930821418762, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.16}


  7%|▋         | 200/3065 [01:55<27:07,  1.76it/s]

{'loss': 0.5517, 'grad_norm': 2.5674209594726562, 'learning_rate': 1.6000000000000003e-05, 'epoch': 0.33}


 10%|▉         | 300/3065 [02:51<25:58,  1.77it/s]

...

{'eval_loss': 0.6569569110870361, 'eval_precision': 0.7240038872691934, 'eval_recall': 0.7247081712062257, 'eval_fscore': 0.7243558580456976, 'eval_accuracy': 0.865, 'eval_tp': 745, 'eval_tn': 2888, 'eval_fp': 284, 'eval_fn': 283, 'eval_runtime': 45.3808, 'eval_samples_per_second': 92.55, 'eval_steps_per_second': 5.795, 'epoch': 5.0}


100%|██████████| 3065/3065 [33:17<00:00,  1.88it/s]

{'train_runtime': 1997.2714, 'train_samples_per_second': 24.533, 'train_steps_per_second': 1.535, 'train_loss': 0.23006841447963994, 'epoch': 5.0}


100%|██████████| 3065/3065 [33:18<00:00,  1.53it/s]


{'train_runtime': 1997.2714,
 'train_samples_per_second': 24.533,
 'train_steps_per_second': 1.535,
 'total_flos': 2241109160556000.0,
 'train_loss': 0.23006841447963994,
 'epoch': 5.0}

In [7]:
test = load_test().collect()
test_transformed = transform(test)
deberta_prediction = deberta.predict(load_dataset(test_transformed["text"].to_list(), test_transformed["label"].to_list()))
deberta_prediction

/Users/retipeter/Documents/TU Wien/NLP/nlp-ie-label-legends/src/label_legends/deberta_majority.py:55: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
100%|██████████| 750/750 [02:05<00:00,  5.95it/s]


PredictionOutput(predictions=array([[ 0.02157354, -0.15974028],
       [ 0.16012895, -0.31555876],
       [ 0.16012895, -0.31555876],
       ...,
       [-0.03087621, -0.13163614],
       [-0.03087621, -0.13163614],
       [-0.03087621, -0.13163614]], dtype=float32), label_ids=array([1, 0, 0, ..., 1, 0, 1]), metrics={'test_loss': 0.38887882232666016, 'test_precision': 0.7482574825748257, 'test_recall': 0.5858747993579454, 'test_fscore': 0.6571840115232265, 'test_accuracy': 0.8413333333333334, 'test_tp': 1825, 'test_tn': 8271, 'test_fp': 614, 'test_fn': 1290, 'test_runtime': 126.3444, 'test_samples_per_second': 94.978, 'test_steps_per_second': 5.936})

In [8]:
predictions = DataFrame({"id": test["id"], "label": deberta_prediction.label_ids, "deberta": np.argmax(deberta_prediction.predictions, axis=1)})
predictions.head()

id,label,deberta
i64,i64,i64
100,1,0
10005,0,0
10006,0,0
10007,0,0
10008,0,0


In [10]:
scores_deberta = calculate_scores(predictions["label"], predictions["deberta"])
scores_deberta

precision:	0.7483
recall:		0.5859
fscore:		0.6572
accuracy:	0.8413
tn: 8271	 fp: 614
fn: 1290	 tp: 1825